<a href="https://colab.research.google.com/github/YanGermanoSantos/Criacao_variavel_transacao/blob/main/Variaveis_Explicativas.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

 # Criando Variáveis Explicativas (Feature Engineering)

Em um projeto de Ciência de Dados, ter dados é fundamental, mas ainda mais crucial é ter variáveis de alta qualidade. Variáveis bem escolhidas e transformadas são a chave para modelos de Machine Learning mais precisos e confiáveis. Com base no exercício porposto pela PoD Academy, que envolveu dados de movimentações de compras de clientes, meu objetivo é realizar o processo de criação de variáveis preditivas. Essas variáveis serão usadas no futuro para desenvolver um modelo de Machine Learning robusto, capaz de solucionar eficazmente os desafios e necessidades do negócio.


## Inicializando ambiente spark

In [1]:
!pip install pyspark

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 316.9/316.9 MB 4.5 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
  Created wheel for pyspark: filename=pyspark-3.5.0-py2.py3-none-any.whl size=317425344 sha256=3808064a84d95299d713ab9232723782d558bf5a15103155ef08f385c1baa784
  Stored in directory: /root/.cache/pip/wheels/41/4e/10/c2cf2467f71c678cfc8a6b9ac9241e5e44a01940da8fbb17fc
Successfully built pyspark


In [2]:
from pyspark.sql import SparkSession

In [3]:
# Inicialize a sessão Spark
spark = SparkSession.builder \
    .appName("Feature_Engineering") \
    .getOrCreate()

## 1 - Obtenção dos dados

### 1.1 Transacoes
Em resumo, essa tabela fornece informações sobre as transações de compra realizadas por diferentes clientes, incluindo detalhes sobre o valor da transação, a data e hora em que ocorreram e a categoria à qual a compra se relaciona.

In [4]:
# Lendo CVS das transações em Spark
df_transacoes = spark.read.csv("/content/base_transacoes.csv", header=True, inferSchema=True)
df_transacoes.createOrReplaceTempView("df_transacoes")

**Dicionário de dados transacoes**
* `ID Transação` - Número de identificação único para cada transação no conjunto de dados.
* `ID Cliente` - Número de identificação único para cada cliente.
* `Data` - Data e hora da transação no formato AAAA-MM-DD HH:MM:SS.
* `Valor` - Valor da transação.
* `Categoria` - Categoria da transação, que indica o tipo de compra ou produto adquirido.

In [5]:
# Lendo primeiras linhas
df_transacoes.show()

+------------+----------+--------------------+------------------+-----------+
|ID Transação|ID Cliente|                Data|             Valor|  Categoria|
+------------+----------+--------------------+------------------+-----------+
|           1|         1|2022-11-25 13:50:...|57.287427536330505|   Esportes|
|           2|         1|2020-01-19 12:27:...| 97.07199340552512|  Alimentos|
|           3|         1|2021-12-28 12:33:...|169.10581012381087|     Livros|
|           4|         1|2022-02-05 01:39:...|199.38694865538451|     Roupas|
|           5|         1|2022-11-16 23:06:...|160.00228343317622|Eletrônicos|
|           6|         1|2020-04-25 12:42:...| 9.842481270765422|  Alimentos|
|           7|         1|2022-10-31 22:05:...| 76.90706330227667|Eletrônicos|
|           8|         1|2020-10-06 12:14:...| 53.20607404593595|     Roupas|
|           9|         1|2022-11-02 12:50:...|193.00568791656067|     Livros|
|          10|         1|2020-09-06 21:37:...|56.524638713138636

In [6]:
# Verificando tipos dos dados
df_transacoes.printSchema()

root
 |-- ID Transação: integer (nullable = true)
 |-- ID Cliente: integer (nullable = true)
 |-- Data: timestamp (nullable = true)
 |-- Valor: double (nullable = true)
 |-- Categoria: string (nullable = true)



### 1.2 Publico
Esta tabela apresenta uma visão geral das ações e características dos clientes.

In [25]:
# Lendo CVS do publico em Spark
df_publico = spark.read.csv("/content/base_churn.csv", header=True, inferSchema=True)
df_publico.createOrReplaceTempView("df_publico")

**Dicionário de dados publico**
* `ID` - Número de identificação único para cada cliente.
* `Idade` - Idade do cliente.
* `Gênero` - Gênero do cliente (por exemplo, 'F' para feminino, 'M' para masculino).
* `Dias desde a Inscrição` - O número de dias desde que o cliente se inscreveu.
* `Usou Suporte` - Indica se o cliente usou o suporte (1 para sim, 0 para não).
* `Plano` - O tipo de plano de serviço ao qual o cliente está inscrito (por exemplo, 'Intermediário', 'Básico').
* `Churn` - Indica se o cliente saiu ou não (1 para saiu, 0 para não saiu).






In [8]:
# Lendo primeiras linhas
df_publico.show()

+---+-----+------+----------------------+------------+-------------+-----+
| ID|Idade|Gênero|Dias desde a Inscrição|Usou Suporte|        Plano|Churn|
+---+-----+------+----------------------+------------+-------------+-----+
|  1|   21|     F|                  1331|           1|Intermediário|    1|
|  2|   21|     M|                  1160|           0|Intermediário|    0|
|  3|   62|     M|                   454|           1|       Básico|    0|
|  4|   64|     M|                   226|           1|Intermediário|    0|
|  5|   61|     M|                   474|           1|     Avançado|    0|
|  6|   18|     M|                   419|           0|       Básico|    0|
|  7|   52|     M|                  1334|           0|       Básico|    0|
|  8|   44|     M|                  1124|           1|Intermediário|    0|
|  9|   52|     M|                  1256|           1|Intermediário|    0|
| 10|   64|     F|                  1197|           0|       Básico|    0|
| 11|   50|     F|       

In [9]:
# Verificando tipos dos dados
df_publico.printSchema()

root
 |-- ID: integer (nullable = true)
 |-- Idade: integer (nullable = true)
 |-- Gênero: string (nullable = true)
 |-- Dias desde a Inscrição: integer (nullable = true)
 |-- Usou Suporte: integer (nullable = true)
 |-- Plano: string (nullable = true)
 |-- Churn: integer (nullable = true)



## 2 Criação de variáveis
Como mencionado anteriormente, o objetivo deste trabalho é criar variáveis que melhorem o desempenho do nosso algoritmo. Para isso, vamos gerar diversas variáveis por meio de agregações, uma vez que não podemos ter linhas duplicadas para cada cliente, pois isso pode afetar a modelagem de machine learning.

### 2.1 Variável tempo
Uma das variáveis essenciais que pretendemos criar é a variável de tempo. Ao agregarmos dados de dias anteriores, poderemos entender melhor o comportamento de compra dos clientes ao longo dos meses passados. Isso nos permitirá identificar tendências sazonais e padrões de compra, o que é crucial para otimizar estratégias de marketing e retenção de clientes. Além disso, essa variável de tempo também será valiosa para previsões futuras, ajudando-nos a antecipar as necessidades dos clientes e personalizar nossas ofertas de acordo com seu histórico de compras. Portanto, criar essa variável de tempo é um passo importante para melhorar o desempenho de nosso algoritmo de machine learning e impulsionar os resultados do negócio.

In [19]:
df_transacoes_tempo = spark.sql("""
SELECT
    *,
    Data,
    CASE
        WHEN Data BETWEEN DATE_ADD(MAX(Data) OVER (PARTITION BY `ID Cliente`), -90) AND MAX(Data) OVER (PARTITION BY `ID Cliente`) THEN 1
        ELSE 0
    END AS ultimos_3_meses_flag,
    CASE
        WHEN Data BETWEEN DATE_ADD(MAX(Data) OVER (PARTITION BY `ID Cliente`), -180) AND MAX(Data) OVER (PARTITION BY `ID Cliente`) THEN 1
        ELSE 0
    END AS ultimos_6_meses_flag,
    CASE
        WHEN Data BETWEEN DATE_ADD(MAX(Data) OVER (PARTITION BY `ID Cliente`), -365) AND MAX(Data) OVER (PARTITION BY `ID Cliente`) THEN 1
        ELSE 0
    END AS ultimos_12_meses_flag
FROM df_transacoes
ORDER BY `ID Cliente`;
""")
df_transacoes_tempo.createOrReplaceTempView("df_transacoes_tempo")

### 2.2 Criando variáveis / Categórica, tempo e númerica
Vamos criar um loop para a criação de agregações em uma consulta, permitindo-nos montar uma consulta para cada variável categórica existente em nossa base. Além de construir mais de 50 variáveis de forma rápida

In [11]:
# Agregações
agg = ['SUM', 'MIN', 'MAX', 'AVG']

In [12]:
# Variável
cat = ['Categoria']

In [13]:
# saída
dominios = ['Livros', 'Eletrônicos', 'Alimentos', 'Esportes', 'Roupas']

In [14]:
# Númerica
var_num = ['Valor']

In [15]:
# Tempo
meses = ['ultimos_3_meses_flag', 'ultimos_6_meses_flag', 'ultimos_12_meses_flag']

In [16]:
for ag in agg:
  for cate in cat:
    if cate == 'Categoria':
      for dominio in dominios:
        for var_n in var_num:
          print(ag + '(CASE WHEN ' + cate + '= ' + "'" + dominio + "'" + ' THEN ' + var_n + ' ELSE 0 END) AS ' + ag + "_" + cate + "_" + dominio.replace('ô', 'o') + "_" + var_n + ",")
        for mes in meses:
          if mes == 'ultimos_3_meses_flag':
            print(ag + '(CASE WHEN ' + cate + '= ' + "'" + dominio + "'" " AND " + mes + '= 1' + ' THEN ' + var_n + ' ELSE NULL END) AS ' + ag + "_" + cate + "_" + dominio.replace('ô', 'o') + "_" + var_n + mes + ",")
          if mes == 'ultimos_6_meses_flag':
            print(ag + '(CASE WHEN ' + cate + '= ' + "'" + dominio + "'" " AND " + mes + '= 1' + ' THEN ' + var_n + ' ELSE NULL END) AS ' + ag + "_" + cate + "_" + dominio.replace('ô', 'o') + "_" + var_n + mes + ",")
          if mes == 'ultimos_12_meses_flag':
            print(ag + '(CASE WHEN ' + cate + '= ' + "'" + dominio + "'" " AND " + mes + '= 1' + ' THEN ' + var_n + ' ELSE NULL END) AS ' + ag + "_" + cate + "_" + dominio.replace('ô', 'o') + "_" + var_n + mes + ",")

SUM(CASE WHEN Categoria= 'Livros' THEN Valor ELSE 0 END) AS SUM_Categoria_Livros_Valor,
SUM(CASE WHEN Categoria= 'Livros' AND ultimos_3_meses_flag= 1 THEN Valor ELSE NULL END) AS SUM_Categoria_Livros_Valorultimos_3_meses_flag,
SUM(CASE WHEN Categoria= 'Livros' AND ultimos_6_meses_flag= 1 THEN Valor ELSE NULL END) AS SUM_Categoria_Livros_Valorultimos_6_meses_flag,
SUM(CASE WHEN Categoria= 'Livros' AND ultimos_12_meses_flag= 1 THEN Valor ELSE NULL END) AS SUM_Categoria_Livros_Valorultimos_12_meses_flag,
SUM(CASE WHEN Categoria= 'Eletrônicos' THEN Valor ELSE 0 END) AS SUM_Categoria_Eletronicos_Valor,
SUM(CASE WHEN Categoria= 'Eletrônicos' AND ultimos_3_meses_flag= 1 THEN Valor ELSE NULL END) AS SUM_Categoria_Eletronicos_Valorultimos_3_meses_flag,
SUM(CASE WHEN Categoria= 'Eletrônicos' AND ultimos_6_meses_flag= 1 THEN Valor ELSE NULL END) AS SUM_Categoria_Eletronicos_Valorultimos_6_meses_flag,
SUM(CASE WHEN Categoria= 'Eletrônicos' AND ultimos_12_meses_flag= 1 THEN Valor ELSE NULL END) AS 

### 2.3 Contruindo a query
Vamos adicionar a saída do looping feito acima e adiciona em uma query

In [30]:
df_transacoes_agg = spark.sql("""
SELECT
  `ID Cliente`,
  SUM(CASE WHEN Categoria= 'Livros' THEN Valor ELSE 0 END) AS SUM_Categoria_Livros_Valor,
  SUM(CASE WHEN Categoria= 'Livros' AND ultimos_3_meses_flag= 1 THEN Valor ELSE NULL END) AS SUM_Categoria_Livros_Valorultimos_3_meses_flag,
  SUM(CASE WHEN Categoria= 'Livros' AND ultimos_6_meses_flag= 1 THEN Valor ELSE NULL END) AS SUM_Categoria_Livros_Valorultimos_6_meses_flag,
  SUM(CASE WHEN Categoria= 'Livros' AND ultimos_12_meses_flag= 1 THEN Valor ELSE NULL END) AS SUM_Categoria_Livros_Valorultimos_12_meses_flag,
  SUM(CASE WHEN Categoria= 'Eletrônicos' THEN Valor ELSE 0 END) AS SUM_Categoria_Eletronicos_Valor,
  SUM(CASE WHEN Categoria= 'Eletrônicos' AND ultimos_3_meses_flag= 1 THEN Valor ELSE NULL END) AS SUM_Categoria_Eletronicos_Valorultimos_3_meses_flag,
  SUM(CASE WHEN Categoria= 'Eletrônicos' AND ultimos_6_meses_flag= 1 THEN Valor ELSE NULL END) AS SUM_Categoria_Eletronicos_Valorultimos_6_meses_flag,
  SUM(CASE WHEN Categoria= 'Eletrônicos' AND ultimos_12_meses_flag= 1 THEN Valor ELSE NULL END) AS SUM_Categoria_Eletronicos_Valorultimos_12_meses_flag,
  SUM(CASE WHEN Categoria= 'Alimentos' THEN Valor ELSE 0 END) AS SUM_Categoria_Alimentos_Valor,
  SUM(CASE WHEN Categoria= 'Alimentos' AND ultimos_3_meses_flag= 1 THEN Valor ELSE NULL END) AS SUM_Categoria_Alimentos_Valorultimos_3_meses_flag,
  SUM(CASE WHEN Categoria= 'Alimentos' AND ultimos_6_meses_flag= 1 THEN Valor ELSE NULL END) AS SUM_Categoria_Alimentos_Valorultimos_6_meses_flag,
  SUM(CASE WHEN Categoria= 'Alimentos' AND ultimos_12_meses_flag= 1 THEN Valor ELSE NULL END) AS SUM_Categoria_Alimentos_Valorultimos_12_meses_flag,
  SUM(CASE WHEN Categoria= 'Esportes' THEN Valor ELSE 0 END) AS SUM_Categoria_Esportes_Valor,
  SUM(CASE WHEN Categoria= 'Esportes' AND ultimos_3_meses_flag= 1 THEN Valor ELSE NULL END) AS SUM_Categoria_Esportes_Valorultimos_3_meses_flag,
  SUM(CASE WHEN Categoria= 'Esportes' AND ultimos_6_meses_flag= 1 THEN Valor ELSE NULL END) AS SUM_Categoria_Esportes_Valorultimos_6_meses_flag,
  SUM(CASE WHEN Categoria= 'Esportes' AND ultimos_12_meses_flag= 1 THEN Valor ELSE NULL END) AS SUM_Categoria_Esportes_Valorultimos_12_meses_flag,
  SUM(CASE WHEN Categoria= 'Roupas' THEN Valor ELSE 0 END) AS SUM_Categoria_Roupas_Valor,
  SUM(CASE WHEN Categoria= 'Roupas' AND ultimos_3_meses_flag= 1 THEN Valor ELSE NULL END) AS SUM_Categoria_Roupas_Valorultimos_3_meses_flag,
  SUM(CASE WHEN Categoria= 'Roupas' AND ultimos_6_meses_flag= 1 THEN Valor ELSE NULL END) AS SUM_Categoria_Roupas_Valorultimos_6_meses_flag,
  SUM(CASE WHEN Categoria= 'Roupas' AND ultimos_12_meses_flag= 1 THEN Valor ELSE NULL END) AS SUM_Categoria_Roupas_Valorultimos_12_meses_flag,
  MIN(CASE WHEN Categoria= 'Livros' THEN Valor ELSE 0 END) AS MIN_Categoria_Livros_Valor,
  MIN(CASE WHEN Categoria= 'Livros' AND ultimos_3_meses_flag= 1 THEN Valor ELSE NULL END) AS MIN_Categoria_Livros_Valorultimos_3_meses_flag,
  MIN(CASE WHEN Categoria= 'Livros' AND ultimos_6_meses_flag= 1 THEN Valor ELSE NULL END) AS MIN_Categoria_Livros_Valorultimos_6_meses_flag,
  MIN(CASE WHEN Categoria= 'Livros' AND ultimos_12_meses_flag= 1 THEN Valor ELSE NULL END) AS MIN_Categoria_Livros_Valorultimos_12_meses_flag,
  MIN(CASE WHEN Categoria= 'Eletrônicos' THEN Valor ELSE 0 END) AS MIN_Categoria_Eletronicos_Valor,
  MIN(CASE WHEN Categoria= 'Eletrônicos' AND ultimos_3_meses_flag= 1 THEN Valor ELSE NULL END) AS MIN_Categoria_Eletronicos_Valorultimos_3_meses_flag,
  MIN(CASE WHEN Categoria= 'Eletrônicos' AND ultimos_6_meses_flag= 1 THEN Valor ELSE NULL END) AS MIN_Categoria_Eletronicos_Valorultimos_6_meses_flag,
  MIN(CASE WHEN Categoria= 'Eletrônicos' AND ultimos_12_meses_flag= 1 THEN Valor ELSE NULL END) AS MIN_Categoria_Eletronicos_Valorultimos_12_meses_flag,
  MIN(CASE WHEN Categoria= 'Alimentos' THEN Valor ELSE 0 END) AS MIN_Categoria_Alimentos_Valor,
  MIN(CASE WHEN Categoria= 'Alimentos' AND ultimos_3_meses_flag= 1 THEN Valor ELSE NULL END) AS MIN_Categoria_Alimentos_Valorultimos_3_meses_flag,
  MIN(CASE WHEN Categoria= 'Alimentos' AND ultimos_6_meses_flag= 1 THEN Valor ELSE NULL END) AS MIN_Categoria_Alimentos_Valorultimos_6_meses_flag,
  MIN(CASE WHEN Categoria= 'Alimentos' AND ultimos_12_meses_flag= 1 THEN Valor ELSE NULL END) AS MIN_Categoria_Alimentos_Valorultimos_12_meses_flag,
  MIN(CASE WHEN Categoria= 'Esportes' THEN Valor ELSE 0 END) AS MIN_Categoria_Esportes_Valor,
  MIN(CASE WHEN Categoria= 'Esportes' AND ultimos_3_meses_flag= 1 THEN Valor ELSE NULL END) AS MIN_Categoria_Esportes_Valorultimos_3_meses_flag,
  MIN(CASE WHEN Categoria= 'Esportes' AND ultimos_6_meses_flag= 1 THEN Valor ELSE NULL END) AS MIN_Categoria_Esportes_Valorultimos_6_meses_flag,
  MIN(CASE WHEN Categoria= 'Esportes' AND ultimos_12_meses_flag= 1 THEN Valor ELSE NULL END) AS MIN_Categoria_Esportes_Valorultimos_12_meses_flag,
  MIN(CASE WHEN Categoria= 'Roupas' THEN Valor ELSE 0 END) AS MIN_Categoria_Roupas_Valor,
  MIN(CASE WHEN Categoria= 'Roupas' AND ultimos_3_meses_flag= 1 THEN Valor ELSE NULL END) AS MIN_Categoria_Roupas_Valorultimos_3_meses_flag,
  MIN(CASE WHEN Categoria= 'Roupas' AND ultimos_6_meses_flag= 1 THEN Valor ELSE NULL END) AS MIN_Categoria_Roupas_Valorultimos_6_meses_flag,
  MIN(CASE WHEN Categoria= 'Roupas' AND ultimos_12_meses_flag= 1 THEN Valor ELSE NULL END) AS MIN_Categoria_Roupas_Valorultimos_12_meses_flag,
  MAX(CASE WHEN Categoria= 'Livros' THEN Valor ELSE 0 END) AS MAX_Categoria_Livros_Valor,
  MAX(CASE WHEN Categoria= 'Livros' AND ultimos_3_meses_flag= 1 THEN Valor ELSE NULL END) AS MAX_Categoria_Livros_Valorultimos_3_meses_flag,
  MAX(CASE WHEN Categoria= 'Livros' AND ultimos_6_meses_flag= 1 THEN Valor ELSE NULL END) AS MAX_Categoria_Livros_Valorultimos_6_meses_flag,
  MAX(CASE WHEN Categoria= 'Livros' AND ultimos_12_meses_flag= 1 THEN Valor ELSE NULL END) AS MAX_Categoria_Livros_Valorultimos_12_meses_flag,
  MAX(CASE WHEN Categoria= 'Eletrônicos' THEN Valor ELSE 0 END) AS MAX_Categoria_Eletronicos_Valor,
  MAX(CASE WHEN Categoria= 'Eletrônicos' AND ultimos_3_meses_flag= 1 THEN Valor ELSE NULL END) AS MAX_Categoria_Eletronicos_Valorultimos_3_meses_flag,
  MAX(CASE WHEN Categoria= 'Eletrônicos' AND ultimos_6_meses_flag= 1 THEN Valor ELSE NULL END) AS MAX_Categoria_Eletronicos_Valorultimos_6_meses_flag,
  MAX(CASE WHEN Categoria= 'Eletrônicos' AND ultimos_12_meses_flag= 1 THEN Valor ELSE NULL END) AS MAX_Categoria_Eletronicos_Valorultimos_12_meses_flag,
  MAX(CASE WHEN Categoria= 'Alimentos' THEN Valor ELSE 0 END) AS MAX_Categoria_Alimentos_Valor,
  MAX(CASE WHEN Categoria= 'Alimentos' AND ultimos_3_meses_flag= 1 THEN Valor ELSE NULL END) AS MAX_Categoria_Alimentos_Valorultimos_3_meses_flag,
  MAX(CASE WHEN Categoria= 'Alimentos' AND ultimos_6_meses_flag= 1 THEN Valor ELSE NULL END) AS MAX_Categoria_Alimentos_Valorultimos_6_meses_flag,
  MAX(CASE WHEN Categoria= 'Alimentos' AND ultimos_12_meses_flag= 1 THEN Valor ELSE NULL END) AS MAX_Categoria_Alimentos_Valorultimos_12_meses_flag,
  MAX(CASE WHEN Categoria= 'Esportes' THEN Valor ELSE 0 END) AS MAX_Categoria_Esportes_Valor,
  MAX(CASE WHEN Categoria= 'Esportes' AND ultimos_3_meses_flag= 1 THEN Valor ELSE NULL END) AS MAX_Categoria_Esportes_Valorultimos_3_meses_flag,
  MAX(CASE WHEN Categoria= 'Esportes' AND ultimos_6_meses_flag= 1 THEN Valor ELSE NULL END) AS MAX_Categoria_Esportes_Valorultimos_6_meses_flag,
  MAX(CASE WHEN Categoria= 'Esportes' AND ultimos_12_meses_flag= 1 THEN Valor ELSE NULL END) AS MAX_Categoria_Esportes_Valorultimos_12_meses_flag,
  MAX(CASE WHEN Categoria= 'Roupas' THEN Valor ELSE 0 END) AS MAX_Categoria_Roupas_Valor,
  MAX(CASE WHEN Categoria= 'Roupas' AND ultimos_3_meses_flag= 1 THEN Valor ELSE NULL END) AS MAX_Categoria_Roupas_Valorultimos_3_meses_flag,
  MAX(CASE WHEN Categoria= 'Roupas' AND ultimos_6_meses_flag= 1 THEN Valor ELSE NULL END) AS MAX_Categoria_Roupas_Valorultimos_6_meses_flag,
  MAX(CASE WHEN Categoria= 'Roupas' AND ultimos_12_meses_flag= 1 THEN Valor ELSE NULL END) AS MAX_Categoria_Roupas_Valorultimos_12_meses_flag,
  AVG(CASE WHEN Categoria= 'Livros' THEN Valor ELSE 0 END) AS AVG_Categoria_Livros_Valor,
  AVG(CASE WHEN Categoria= 'Livros' AND ultimos_3_meses_flag= 1 THEN Valor ELSE NULL END) AS AVG_Categoria_Livros_Valorultimos_3_meses_flag,
  AVG(CASE WHEN Categoria= 'Livros' AND ultimos_6_meses_flag= 1 THEN Valor ELSE NULL END) AS AVG_Categoria_Livros_Valorultimos_6_meses_flag,
  AVG(CASE WHEN Categoria= 'Livros' AND ultimos_12_meses_flag= 1 THEN Valor ELSE NULL END) AS AVG_Categoria_Livros_Valorultimos_12_meses_flag,
  AVG(CASE WHEN Categoria= 'Eletrônicos' THEN Valor ELSE 0 END) AS AVG_Categoria_Eletronicos_Valor,
  AVG(CASE WHEN Categoria= 'Eletrônicos' AND ultimos_3_meses_flag= 1 THEN Valor ELSE NULL END) AS AVG_Categoria_Eletronicos_Valorultimos_3_meses_flag,
  AVG(CASE WHEN Categoria= 'Eletrônicos' AND ultimos_6_meses_flag= 1 THEN Valor ELSE NULL END) AS AVG_Categoria_Eletronicos_Valorultimos_6_meses_flag,
  AVG(CASE WHEN Categoria= 'Eletrônicos' AND ultimos_12_meses_flag= 1 THEN Valor ELSE NULL END) AS AVG_Categoria_Eletronicos_Valorultimos_12_meses_flag,
  AVG(CASE WHEN Categoria= 'Alimentos' THEN Valor ELSE 0 END) AS AVG_Categoria_Alimentos_Valor,
  AVG(CASE WHEN Categoria= 'Alimentos' AND ultimos_3_meses_flag= 1 THEN Valor ELSE NULL END) AS AVG_Categoria_Alimentos_Valorultimos_3_meses_flag,
  AVG(CASE WHEN Categoria= 'Alimentos' AND ultimos_6_meses_flag= 1 THEN Valor ELSE NULL END) AS AVG_Categoria_Alimentos_Valorultimos_6_meses_flag,
  AVG(CASE WHEN Categoria= 'Alimentos' AND ultimos_12_meses_flag= 1 THEN Valor ELSE NULL END) AS AVG_Categoria_Alimentos_Valorultimos_12_meses_flag,
  AVG(CASE WHEN Categoria= 'Esportes' THEN Valor ELSE 0 END) AS AVG_Categoria_Esportes_Valor,
  AVG(CASE WHEN Categoria= 'Esportes' AND ultimos_3_meses_flag= 1 THEN Valor ELSE NULL END) AS AVG_Categoria_Esportes_Valorultimos_3_meses_flag,
  AVG(CASE WHEN Categoria= 'Esportes' AND ultimos_6_meses_flag= 1 THEN Valor ELSE NULL END) AS AVG_Categoria_Esportes_Valorultimos_6_meses_flag,
  AVG(CASE WHEN Categoria= 'Esportes' AND ultimos_12_meses_flag= 1 THEN Valor ELSE NULL END) AS AVG_Categoria_Esportes_Valorultimos_12_meses_flag,
  AVG(CASE WHEN Categoria= 'Roupas' THEN Valor ELSE 0 END) AS AVG_Categoria_Roupas_Valor,
  AVG(CASE WHEN Categoria= 'Roupas' AND ultimos_3_meses_flag= 1 THEN Valor ELSE NULL END) AS AVG_Categoria_Roupas_Valorultimos_3_meses_flag,
  AVG(CASE WHEN Categoria= 'Roupas' AND ultimos_6_meses_flag= 1 THEN Valor ELSE NULL END) AS AVG_Categoria_Roupas_Valorultimos_6_meses_flag,
  AVG(CASE WHEN Categoria= 'Roupas' AND ultimos_12_meses_flag= 1 THEN Valor ELSE NULL END) AS AVG_Categoria_Roupas_Valorultimos_12_meses_flag
FROM df_transacoes_tempo
GROUP BY `ID Cliente`
ORDER BY `ID Cliente`
""")
df_transacoes_agg.createOrReplaceTempView("df_transacoes_agg")

In [37]:
# Join com tabela publico
df_transacoes_prep = spark.sql("""
SELECT
*
FROM df_transacoes_agg as ag inner join df_publico as pu on (ag.`ID Cliente` = pu.ID)
""")

In [38]:
# Exportando para um parquet
df_transacoes_prep.write.parquet(path='/content/', mode='overwrite')